In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
data= pd.read_csv('data/EXSA2002108040.csv')

data.columns

In [ ]:
data_1= data[['expression_form', 'expression_score', 'expression_category', 'subject']]
data_1

In [4]:
texts = data_1['expression_form'] # 사용자 발화 내용
labels = data_1['expression_score']  # 감정 라벨

In [ ]:
labels.dtype

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset

# transformer 라이브러리에 맞춰서 Dataset 객체 정의하는 클래스
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)


texts = data_1['expression_form'] # 사용자 발화 내용
labels = data_1['expression_score']  # 감정 라벨

# 학습 데이터셋과 검증 데이터셋으로 분리
X_train, X_test, y_train, y_test=train_test_split(texts, labels, random_state=42)

# 라벨 값 변환 (모델이 이해할 수 있는 숫자로 변환)
label_mapping = {-2: 0, -1: 1, 0: 2, 1: 3, 2: 4}
y_train = y_train.map(label_mapping)
y_test = y_test.map(label_mapping)



# 토크나이저 및 BERT 모델 로드
num_classes = 5 # 감정 라벨이 5개로 분류되어 있음 (0,1,2,3,4 로 분류됨)
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base', num_labels=num_classes)

# 데이터 토크나이징
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)


# 토크나이징한 데이터를 Dataset으로 변환하기 (앞서 정의한 클래스 활용)
train_dataset = EmotionDataset(train_encodings, y_train.tolist())   
test_dataset = EmotionDataset(test_encodings, y_test.tolist())



# trainer 객체를 사용해서 학습시키기
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs=3, # 학습시킬 epoch 수 (epoch 이 많아지면 과적합될 수 있음)
    per_device_train_batch_size=8,  # 학습시킬 배치 사이즈 (배치 사이즈 커지면 GPU 사용량이 많아짐)
    per_device_eval_batch_size=8,  # 검증할때 배치 사이즈
    warmup_steps=500, # 워밍업 스텝 수 (학습 초반 모델이 적응하도록 학습률을 천천히 증가시킴. 학습을 더 안정적으로 만듦)
    weight_decay=0.01, # 가중치 감소 (과적합 방지를 위해 가중치 감소-정규화 시킴)
    logging_dir='./logs',  # 로그 디렉터리 (학습 로그를 기록할 곳)
    logging_steps=10, # 로그 기록할 스텝 수 지정
    evaluation_strategy="steps", # 학습 중 일정 스텝마다 평가하기
    eval_steps=100,  # 평가 스텝 간격
    load_best_model_at_end=True  # 학습 종료 시 최고의 모델 로드
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

In [ ]:
trainer.save_model('./trained_model') # 학습된 모델 저장
tokenizer.save_pretrained('./trained_model') 